<div style="text-align: center; font-weight: bold; font-size: 200%">Projet C2 - Barrier Option in Libor Market Model</div>


In [ ]:
#packages
import numpy as np
import scipy.stats as sps
import sympy as sp
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
from torch.distributions import Normal
import seaborn as sns
import torch
sns.set_theme()
from numpy.random import default_rng, SeedSequence
from scipy.stats import norm

sq = SeedSequence()
rng = default_rng(sq)

In [ ]:
#Bernouilli law with torch
def bernoulli(p, size):
    if size is None:
        size = ()
    binary_samples = torch.bernoulli(torch.full(size, p))
    return 2 * binary_samples - 1

#Returns bernouilli variable when it is given two values and their respectives probabilities
def bernouilli_law(values, probabilities, size):
    sample = np.random.choice(values, size=size, p=probabilities)
    return sample

In [ ]:
#Monte Carlo Method
def monte_carlo(sample, proba):
    mean = np.mean(sample)
    var = np.var(sample, ddof=1)
    alpha = 1 - proba 
    quantile = norm.ppf(1 - alpha/2)
    ci_size = quantile * np.sqrt(var / sample.size)
    return (mean, var, mean - ci_size, mean + ci_size)

<div style="text-align: center; font-weight: bold; font-size: 150%">"Knock-out caplet" pricing<div>

In [ ]:
#Caplet and constant parameters
T=9
K=0.01
h = 0.01
l=0.13
z=0
H=0.28
sigma=0.25

<div style="text-align: center; font-weight: bold; font-size: 100%">Caplet exact price<div>

In [ ]:
#Closed formula
class ExactPrice:

    def __init__(self):
        self.normal_cumulative_function = Normal(torch.tensor(0.0) , torch.tensor(1.0) )

    @classmethod
    def delta_plus(cls, x, v):
        value = (torch.log(x) + 0.5 * v**2) / v
        return value

    @classmethod
    def delta_moins(cls, x, v):
        value = (torch.log(x) - 0.5 * v**2) / v
        return value

    def caplet_closed_formula(self, L0, K, H, v):
        first_term=L0*(self.normal_cumulative_function.cdf(self.delta_plus(L0/K, v))-self.normal_cumulative_function.cdf(self.delta_plus(L0/H, v)))
        second_term=K*(self.normal_cumulative_function.cdf(self.delta_moins(L0/K, v))-self.normal_cumulative_function.cdf(self.delta_moins(L0/H, v)))
        third_term=H*(self.normal_cumulative_function.cdf(self.delta_plus(H**2/(K*L0), v))-self.normal_cumulative_function.cdf(self.delta_plus(H/L0, v)))
        fourth_term=(K*L0/H)*(self.normal_cumulative_function.cdf(self.delta_moins(H**2/(K*L0), v))-self.normal_cumulative_function.cdf(self.delta_moins(H/L0, v)))
        return first_term-second_term-third_term+fourth_term

In [ ]:
#Barrier caplet exact price
v=np.sqrt(sigma**2*T)
exact_price_class = ExactPrice()
exact_price = exact_price_class.caplet_closed_formula(torch.tensor(l), K, H, v)
print(f"real price: {exact_price}")

<div style="text-align: center; font-weight: bold; font-size: 100%">Random walk implementation<div>

In [ ]:
class Utilities:

    def __init__(self):
        self.exact_price_class = ExactPrice()
        self.normal_cumulative_function = Normal(torch.tensor(0.0) , torch.tensor(1.0) )
        
    # Compute F as the derivation of the caplet analytical price
    def F(self, L, K, H, v, sigma):
        first_term=self.normal_cumulative_function.cdf(self.exact_price_class.delta_plus(L/K, v))-self.normal_cumulative_function.cdf(self.exact_price_class.delta_plus(L/H, v))
        fourth_term=(K/H)*(self.normal_cumulative_function.cdf(self.exact_price_class.delta_moins(H**2/(K*L), v))-self.normal_cumulative_function.cdf(self.exact_price_class.delta_moins(H/L, v)))   
        return -sigma*(first_term+fourth_term) 
    """
  
    def F(self, L, K, H, v, sigma):
        epsilon=np.sqrt(np.finfo(float).eps)
        derivative = (self.exact_price_class.caplet_closed_formula(L + epsilon, K, H, v) - self.exact_price_class.caplet_closed_formula(L, K, H, v)) / epsilon
        return -sigma*derivative
    """


In [ ]:
class RandomWalk:

    def __init__(self):
        self.utilities = Utilities()
    
    #Caplet euler scheme
    def libor_diffusion(self, noise, h, l, sigma):
        n, M = noise.shape
        sample = torch.empty((n+1, M))
        sample[0] = np.log(l)
        sample[1:] = - 0.5*h*sigma**2 + sigma*np.sqrt(h)*noise
        sample=torch.cumsum(sample, axis=0)
        return sample

    #Total variance v function
    def __v_function(self, h, M, T, sigma):
        t_i = torch.arange(0, T, h)
        t_i = torch.tile(t_i, (M, 1))
        t_i=t_i.T
        v_sample=torch.sqrt((T-t_i)*sigma**2)
        return v_sample

    #Calcul de Z
    def z_diffusion(self, noise, sample, h, T, z, K, H, sigma):
        n, M = noise.shape
        z_sample = torch.empty((n+1, M))
        z_sample[0] = z
        v_sample=self.__v_function(h, M, T, sigma)
        l_sample=torch.exp(sample[:-1])
        z_sample[1:] = self.utilities.F(l_sample, K, H, v_sample, sigma)*np.sqrt(h)*noise
        z_sample=torch.cumsum(z_sample, axis=0)
        return z_sample
    
    #After diffusing the libor rates this method checks the points which are in the boundary zone
    def boundary_zone(self, sample, H, lambda_h):
        n, M = sample.shape 
        boundary_zone_sample = np.zeros((n, M))
        boundary_zone_sample[np.array(sample)>=np.log(H)-lambda_h] = 1
        max_boundary_zone_sample = np.max(boundary_zone_sample, axis=0)
        col_indices  = np.where(max_boundary_zone_sample == 1)[0]
        ones_matrix= boundary_zone_sample[:, col_indices]
        row_indices = np.argmax(ones_matrix, axis=0)
        return row_indices, col_indices


<div style="text-align: center; font-weight: bold; font-size: 100%">Caplet class<div>

In [ ]:
class Caplet:

    def __init__(self, T, K, l, z, H, sigma):
        self.T=T
        self.K=K
        self.l=l
        self.z=z
        self.H=H
        self.sigma=sigma
        self.rdm_walk = RandomWalk()

    # Compute knock-out caplet sample   
    def caplet(self, xi, h, algorithm, F):
        lambda_h= sigma*np.sqrt(h)
        l_sample=self.rdm_walk.libor_diffusion(xi, h, self.l, self.sigma)
        row_indices, col_indices = self.rdm_walk.boundary_zone(l_sample, H, lambda_h)
        if F:
            z_sample = self.rdm_walk.z_diffusion(xi, l_sample, h, self.T, self.z, self.K, self.H, self.sigma)    
        else: 
            z_sample=None
        caplet_sample = algorithm(l_sample, row_indices, col_indices, z_sample, F, xi, h, self.T,  self.K, self.H, self.sigma, lambda_h)
        return caplet_sample
    
    #Antithetic variance reduction
    def caplet_antithetic_method(self, xi, h, algorithm,  F):
        sample = self.caplet(xi, h, algorithm,  F)
        anti_sample = self.caplet(-xi, h, algorithm,  F)
        return 0.5*(sample+anti_sample)


<div style="text-align: center; font-weight: bold; font-size: 100%">Random walk algorithm 1<div>

In [ ]:
def algorithm_1_explicit_loop_F_False(xi, h, x, sigma, H, K, lambda_h):
    n=len(xi)
    log_H=np.log(H)
    for i in range(n):
        if x>=log_H-lambda_h:
            p=lambda_h/(np.abs(log_H+lambda_h-x))
            value=bernouilli_law([log_H, x-lambda_h], [p, 1-p], 1).item()
            if value==log_H:  
                return 0
            else:
                x=x-lambda_h - 0.5*h*sigma**2 + sigma*np.sqrt(h)*xi[i]
        else:
            x=x- 0.5*h*sigma**2 + sigma*np.sqrt(h)*xi[i]
    else:
        return max(np.exp(x)-K, 0)

In [ ]:
utilities = Utilities()
def algorithm_1_explicit_loop_F_True(xi, T, h, x, z, sigma, H, K, lambda_h):
    n=len(xi)
    log_H=np.log(H)
    for i in range(n):
        t_i = i*h
        v=np.sqrt((T-t_i)*sigma**2)
        if x>=log_H-lambda_h:
            p=lambda_h/(np.abs(log_H+lambda_h-x))
            value=bernouilli_law([log_H, x-lambda_h], [p, 1-p], 1).item()
            if value==log_H: 
                z=z + utilities.F(np.exp(x), K, H, v, sigma)*np.sqrt(h)*xi[i]
                return  z
            else:
                z=z + utilities.F(np.exp(x), K, H, v, sigma)*np.sqrt(h)*xi[i]
                x=x-lambda_h - 0.5*h*sigma**2 + sigma*np.sqrt(h)*xi[i]
        else: 
            z=z + utilities.F(np.exp(x), K, H, v, sigma)*np.sqrt(h)*xi[i]
            x=x - 0.5*h*sigma**2 + sigma*np.sqrt(h)*xi[i]
    return max(np.exp(x)-K, 0)+z

In [ ]:
#To improwe the fastest we use the algorithm 2 (can be vectorized) in first position and after we compute the algorithm 1 (Need explicit loop)
def algorithm_1(l_sample, row_indices, col_indices, z_sample=None, F=True, *args): 
        xi, h, T,  K, H, sigma, lambda_h = args
        final_sample = np.array(l_sample[-1])
        final_sample=np.maximum(np.exp(final_sample)-K, 0)
        final_sample[col_indices]=0
        if F:
            final_z_sample = np.array(z_sample[-1])
            final_z_sample[col_indices]=np.array([algorithm_1_explicit_loop_F_True(xi[i:, j], T-i*h, h, l_sample[i, j], z_sample[i, j], sigma, H, K, lambda_h) for i, j in zip(row_indices, col_indices)])
            return final_sample+final_z_sample
        else: 
            final_sample[col_indices]=np.array([algorithm_1_explicit_loop_F_False(xi[i:, j], h, l_sample[i, j], sigma, H, K, lambda_h) for i, j in zip(row_indices, col_indices)])
            return final_sample

<div style="text-align: center; font-weight: bold; font-size: 100%">Random walk algorithm 2<div>

In [ ]:
def algorithm_2(l_sample, row_indices, col_indices, z_sample=None, F=True, *args): 
    xi, h, T,  K, H, sigma, lambda_h = args
    final_sample = l_sample[-1]
    final_sample=np.maximum(np.exp(final_sample)-K, 0)
    final_sample[col_indices]=0
    if F:
        final_z_sample = np.array(z_sample[-1])
        indices=list(zip(row_indices, col_indices))
        final_z_sample[col_indices] = np.array([z_sample[idx] for idx in indices])
        return final_sample+final_z_sample
    else:
        return final_sample

<div style="text-align: center; font-weight: bold; font-size: 100%">"Knock-out" caplet pricing example<div>

Execution time of the two algorithms

In [ ]:
n=int(np.ceil(T/h))
M=100
xi=bernoulli(0.5, (n, M))
caplet  = Caplet(T, K, l, z, H, sigma)
%timeit caplet.caplet(xi, h, algorithm_1, F=True)
%timeit caplet.caplet(xi, h, algorithm_2, F=True)

Algorithm 1 is very expensive in execution time. For the different comparisons between the two algorithms we will take M=10000.

For the numerical study we will focus on algorithm 2 and we will take M=100000

In [ ]:
n=int(np.ceil(T/h))
M=10000
xi=bernoulli(0.5, (n, M))
caplet  = Caplet(T, K, l, z, H, sigma)
results=pd.DataFrame(columns=["Mean", "Var", "Low", "Up"])

In [ ]:
#Algorithm 1
sample=caplet.caplet(xi, h, algorithm_1, F=False)
results.loc[r"Algorithm 1 F=0"] =monte_carlo(np.array(sample), 0.95)
sample=caplet.caplet(xi, h, algorithm_1, F=True)
results.loc[r"Algorithm 1 F!=0"] =monte_carlo(np.array(sample), 0.95)

In [ ]:
#Algorithm 2
sample=caplet.caplet(xi, h, algorithm_2, F=False)
results.loc[r"Algorithm 2 F=0"] =monte_carlo(np.array(sample), 0.95)
sample=caplet.caplet(xi, h, algorithm_2, F=True)
results.loc[r"Algorithm 2 F!= 0"] =monte_carlo(np.array(sample), 0.95)

In [ ]:
results

<div style="text-align: center; font-weight: bold; font-size: 100%">Variance reduction with the antithetic method<div>

In [ ]:
anti_results=pd.DataFrame(columns=["Mean", "Var", "Low", "Up"])

In [ ]:
#Algorithm 1
sample=caplet.caplet_antithetic_method(xi, h, algorithm_1, F=False)
anti_results.loc[r"Algorithm 1 F=0"] =monte_carlo(np.array(sample), 0.95)
sample=caplet.caplet_antithetic_method(xi, h, algorithm_1, F=True)
anti_results.loc[r"Algorithm 1 F!=0"] =monte_carlo(np.array(sample), 0.95)

In [ ]:
#Algorithm 2
sample=caplet.caplet_antithetic_method(xi, h, algorithm_2, F=False)
anti_results.loc[r"Algorithm 2 F=0"] =monte_carlo(np.array(sample), 0.95)
sample=caplet.caplet_antithetic_method(xi, h, algorithm_2, F=True)
anti_results.loc[r"Algorithm 2 F!= 0"] =monte_carlo(np.array(sample), 0.95)

In [ ]:
anti_results

<div style="text-align: center; font-weight: bold; font-size: 100%">Graph visualizing the pricing error as a function of step h<div>

In [ ]:
M=100000

In [ ]:
def mean_algorithm_2(iters, T , M, F=False):
    means=np.empty((len(iters), 4))
    for i in range(len(iters)):
        h=iters[i]
        n=int(np.ceil(T/h))
        xi=bernoulli(0.5, (n, M))
        sample=caplet.caplet_antithetic_method(xi, h, algorithm_2, F)
        means[i,:]=monte_carlo(np.array(sample), 0.95)
    return means

In [ ]:
iters = np.arange(0.01, 0.1, 0.005)
rng_algorithm_2_f_true=mean_algorithm_2(iters, T , M, F=True)
rng_algorithm_2_f_False=mean_algorithm_2(iters, T , M, F=False)

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(9, 4), ncols=2, nrows=1, sharey=False, layout='tight')

ax1.plot(iters, rng_algorithm_2_f_False[:,0], label="MC estimator")
ax1.fill_between(iters, rng_algorithm_2_f_False[:,2], rng_algorithm_2_f_False[:,3], facecolor='lightyellow', 
                    label="95% CI")
ax1.set_title(r"Algorithm 2 avec F = 0")

ax2.plot(iters, rng_algorithm_2_f_true[:,0], label="MC estimator")
ax2.fill_between(iters, rng_algorithm_2_f_true[:,2], rng_algorithm_2_f_true[:,3], facecolor='lightyellow', 
                    label="95% CI")
ax2.set_title(r"Algorithm 2 avec F $\neq$ 0")

ax1.plot(iters, [exact_price]*len(iters), label="Exact Price")
ax2.plot(iters, [exact_price]*len(iters), label="Exact Price")

ax1.legend()
ax2.legend(loc='lower left')

plt.show()

<div style="text-align: center; font-weight: bold; font-size: 100%">Richardson extrapolation: Helps reduce bias<div>

In [ ]:
def richardson_mean_algorithm_2(iters, T , M, F=False):
    means=np.empty((len(iters), 4))
    for i in range(len(iters)):
        h=iters[i]
        N=int(np.ceil(T/h))
        xi=bernoulli(0.5, (N, M))
        sample_N=caplet.caplet_antithetic_method(xi, h, algorithm_2, F)
        
        n=int(np.ceil(T/(2*h)))
        xi=bernoulli(0.5, (n, M))
        sample_n=caplet.caplet_antithetic_method(xi, 2*h, algorithm_2, F)
        means[i,:]=monte_carlo(np.array(2*sample_N-sample_n), 0.95)
    return means

In [ ]:
iters = np.arange(0.01, 0.1, 0.005)
richardson_rng_algorithm_2_f_true=richardson_mean_algorithm_2(iters, T , M, F=True)
richardson_rng_algorithm_2_f_False=richardson_mean_algorithm_2(iters, T , M, F=False)

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(9, 4), ncols=2, nrows=1, sharey=False, layout='tight')

ax1.plot(iters, richardson_rng_algorithm_2_f_False[:,0], label="MC estimator")
ax1.fill_between(iters, richardson_rng_algorithm_2_f_False[:,2], richardson_rng_algorithm_2_f_False[:,3], facecolor='lightyellow', 
                    label="95% CI")
ax1.set_title(r"Richardson Algorithm 2 avec F = 0")

ax2.plot(iters, richardson_rng_algorithm_2_f_true[:,0], label="MC estimator")
ax2.fill_between(iters, richardson_rng_algorithm_2_f_true[:,2], richardson_rng_algorithm_2_f_true[:,3], facecolor='lightyellow', 
                    label="95% CI")
ax2.set_title(r"Richardson Algorithm 2 avec F $\neq$ 0")

ax1.plot(iters, [exact_price]*len(iters), label="Exact Price")
ax2.plot(iters, [exact_price]*len(iters), label="Exact Price")

ax1.legend()
ax2.legend(loc='lower left')

plt.show()

<div style="text-align: center; font-weight: bold; font-size: 100%">Pricing of the caplet with the diffusion bridge method<div>

In [ ]:
class CapletBrownianBridge:

    def __init__(self, T, K, l, H, sigma):
        self.T=T
        self.K=K
        self.l=l
        self.H=H
        self.sigma=sigma
        self.rdm_walk = RandomWalk()

    def __indicator_function(self, lsample):
        max_vector = np.max(lsample, axis=0)
        indicator_vector = max_vector<=np.log(self.H)
        return indicator_vector.astype(int)


    # Compute knock-out caplet sample   
    def caplet(self, xi, h):
        n, M = xi.shape
        l_sample=np.array(self.rdm_walk.libor_diffusion(xi, h, self.l, self.sigma))
        payoff = np.maximum(np.exp(l_sample[-1])-self.K, 0)
        indicator_vector = self.__indicator_function(l_sample)
        probabilities_term = np.prod(1-np.exp(-(2*n/T)*((np.exp(l_sample[:-1])-self.H)*(np.exp(l_sample[1:])-self.H))/self.sigma**2), axis=0)
        caplet_sample = payoff*indicator_vector*probabilities_term
        return caplet_sample
    
    #Antithetic variance reduction
    def caplet_antithetic_method(self, xi, h):
        sample = self.caplet(xi, h)
        anti_sample = self.caplet(-xi, h)
        return 0.5*(sample+anti_sample)

In [ ]:
n=int(np.ceil(T/h))
M=100000
caplet_brownian_bridge  = CapletBrownianBridge(T, K, l, H, sigma)
xi=torch.normal(0, 1, (n, M))
results=pd.DataFrame(columns=["Mean", "Var", "Low", "Up"])

In [ ]:
#Algorithm 2
sample=caplet_brownian_bridge.caplet_antithetic_method(xi, h)
results.loc["Bridge algorithm"] =monte_carlo(np.array(sample), 0.95)

In [ ]:
results

In [ ]:
def brownian_bridge_mean_algorithm(iters, T , M):
    means=np.empty((len(iters), 4))
    for i in range(len(iters)):
        h=iters[i]
        n=int(np.ceil(T/h))
        xi=torch.normal(0, 1, (n, M))
        sample=caplet_brownian_bridge.caplet_antithetic_method(xi, h)
        means[i,:]=monte_carlo(np.array(sample), 0.95)
    return means

In [ ]:
iters = np.arange(0.01, 0.1, 0.005)
brownian_bridge_rng=brownian_bridge_mean_algorithm(iters, T , M)

In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(9, 4), ncols=2, nrows=1, sharey=False, layout='tight')
ax1.plot(iters, rng_algorithm_2_f_False[:,0], label="MC estimator")
ax1.fill_between(iters, rng_algorithm_2_f_False[:,2], rng_algorithm_2_f_False[:,3], facecolor='lightyellow', 
                    label="95% CI")
ax1.set_title(r"Algorithm 2 avec F = 0")
ax2.plot(iters, brownian_bridge_rng[:,0], label="MC estimator")
ax2.fill_between(iters, brownian_bridge_rng[:,2], brownian_bridge_rng[:,3], facecolor='lightyellow', 
                    label="95% CI")
ax2.set_title(r"Brownian Bridge")

ax1.plot(iters, [exact_price]*len(iters), label="Exact Price")
ax2.plot(iters, [exact_price]*len(iters), label="Exact Price")

ax1.legend()
ax2.legend(loc='lower left')

plt.show()

<div style="text-align: center; font-weight: bold; font-size: 150%">"Knock-out swaption" pricing<div>

In [ ]:
#Swaption  and constant parameters
T0=10
T_N=20
K=0.01
delta=1
h = 0.01
L0=0.05
Rup=0.075
sigma=0.1
beta=0.1

<div style="text-align: center; font-weight: bold; font-size: 100%">Swaption class<div>

In [ ]:
class Swaption:

    def __init__(self, T0, T_N, K, L0, delta, Rup, sigma, beta):
        self.T0=T0
        self.T_N=T_N
        self.N = self.T_N-self.T0
        self.K=K
        self.delta=delta
        self.L= np.full(self.N, L0)
        self.Rup=Rup
        self.Sigma = np.full(self.N, sigma)
        self.beta=beta
        rho=np.empty((self.N, self.N))
        T_i = np.arange(self.T0, self.T_N)
        T_i = np.tile(T_i, (self.N, 1))
        self.rho = np.exp(-self.beta*np.abs(T_i-T_i.T))
        self.U = np.linalg.cholesky(self.rho)

    def swaption(self, xi, h, algorithm):
        xi_shape = xi.shape
        M=xi_shape[2]
        sigma_max = np.max(self.Sigma)
        lambda_h = np.sqrt(self.N)*(sigma_max**2*h*self.N-0.5*sigma_max**2*h+sigma_max*np.sqrt(h*self.N))
        swaption_sample = [algorithm(xi[:, :, i], h, self.T0, self.L, self.Sigma, self.Rup, self.K, self.delta, self.rho, self.U, lambda_h) for i in range(M)]
        return np.array(swaption_sample)

    #Antithetic variance reduction
    def swaption_antithetic_method(self, xi, h, algorithm):
        sample = self.swaption(xi, h, algorithm)
        anti_sample = self.swaption(-xi, h, algorithm)
        return 0.5*(sample+anti_sample)

In [ ]:
swaption = Swaption(T0, T_N, K, L0, delta, Rup, sigma, beta)

<div style="text-align: center; font-weight: bold; font-size: 100%">Swaption proxy price<div>

In [ ]:
class SwaptionUtilities: 

    def __init__(self):
        pass

    @classmethod
    def libor_product_inverse(cls, delta, L):
        return 1/np.prod(delta*L+1)      
      
    @classmethod
    def Rswap_denominator(cls, delta, L):
        denominator_sum=0
        for i in range(len(L)):
            denominator_sum+= cls.libor_product_inverse(delta, L[:i+1])
        return delta*denominator_sum             

    @classmethod
    def Rswap(cls, delta, L, denominator_sum):
        numerator=1-cls.libor_product_inverse(delta, L)
        return numerator/denominator_sum
    
    @classmethod
    def omega(cls, delta, Li, denominator_sum):
        numerator=delta*cls.libor_product_inverse(delta, Li)
        return numerator/denominator_sum

In [ ]:
#Closed formula
def delta_plus(x, v):
    return (np.log(x)+0.5*v**2)/v

def delta_moins(x, v):
    return (np.log(x)-0.5*v**2)/v

def v_swap(delta, L, T, sigma, rho, rswap, denominator_sum):
    constant_term = T/(rswap**2)
    sum_term=0
    for i in range(len(L)):
        for j in  range(len(L)):
            sum_term+= SwaptionUtilities.omega(delta, L[:i+1], denominator_sum)*SwaptionUtilities.omega(delta, L[:j+1], denominator_sum)*L[i]*L[j]*sigma[i]*sigma[j]*rho[i,j]
    return np.sqrt(constant_term*sum_term)

def swaption_closed_formula(delta, L, K, Rup, sigma, rho, T0):
    denominator_sum = SwaptionUtilities.Rswap_denominator(delta, L)
    zc= (1/(1+L[0])**T0)
    rswap = SwaptionUtilities.Rswap(delta, L, denominator_sum)
    v=v_swap(delta, L, T0, sigma, rho, rswap, denominator_sum)
    first_term=rswap*(norm.cdf(delta_plus(rswap/K, v))-norm.cdf(delta_plus(rswap/Rup, v)))
    second_term=K*(norm.cdf(delta_moins(rswap/K, v))-norm.cdf(delta_moins(rswap/Rup, v)))
    third_term=Rup*(norm.cdf(delta_plus(Rup**2/(K*rswap), v))-norm.cdf(delta_plus(Rup/rswap, v)))
    fourth_term=(K*rswap/Rup)*(norm.cdf(delta_moins(Rup**2/(K*rswap), v))-norm.cdf(delta_moins(Rup/rswap, v)))
    swaption_price=(first_term-second_term-third_term+fourth_term)*denominator_sum*zc
    return swaption_price

In [ ]:
#Barrier caplet exact price
proxy_price = swaption_closed_formula(delta, swaption.L, K, Rup, swaption.Sigma, swaption.rho, T0)
print(f"proxy price: {proxy_price}")

<div style="text-align: center; font-weight: bold; font-size: 100%">Random walk algorithm 1<div>

In [ ]:
#L projection on the boundary zone

def objective(projected_L, L):
    return np.sum((L-projected_L)**2)

def L_projection(L, L0, Rup, delta):
    constraint = lambda x: np.log(((Rup*(1+np.sum([np.prod(delta*np.exp(x[i:])+1) for i in range(1, len(L))]))+1)/(np.prod(delta*np.exp(x[1:])+1)))-(1/delta))-x[0]
    constraint_eq = {'type': 'eq', 'fun': constraint}
    result = minimize(objective, L0, args=L, constraints=constraint_eq)
    return result.x, result.fun 

In [ ]:
def swaption_algorithm_1(xi, h, T0, L, Sigma, Rup, K, delta, rho, U, lambda_h):
    n, N  = xi.shape
    boundary_zone_term = np.log(Rup)-(lambda_h/np.sqrt(N))
    zc= (1/(1+L[0])**T0)
    L=np.log(L)
    sigma_max=np.max(Sigma)
    increasing_N = np.arange(1, N+1)
    decreasing_N = np.arange(N, 0, -1)
    for i in range(n):
        boundary_zone_L = np.exp(L)*(1+increasing_N*sigma_max*Sigma*h+Sigma*np.sqrt(decreasing_N*h))
        denominator_sum = SwaptionUtilities.Rswap_denominator(delta, boundary_zone_L)
        boundary_zone_rswap = SwaptionUtilities.Rswap(delta, boundary_zone_L, denominator_sum)
        if np.max(L)>=boundary_zone_term and boundary_zone_rswap>=Rup:  
            projected_L, squared_sum = L_projection(L, np.log(boundary_zone_L), Rup, delta)
            p=lambda_h/(np.sqrt(squared_sum)+lambda_h)
            u=np.random.uniform(0.5, 1, 1)
            if u<p: 
                return 0
            else:
                interior_L = L + lambda_h*(L-projected_L)/np.sqrt(squared_sum)
                for j in range(N):
                    drift=Sigma[j]*np.sum((delta*interior_L[:j+1])/(1+delta*interior_L[:j+1])*rho[j,:j+1]*Sigma[:j+1]*h)
                    L[j]= interior_L[j] + drift -0.5*Sigma[j]**2*h + Sigma[j]* np.sqrt(h)*np.sum(U[j,:]*xi[i]) 
        else:
            for j in range(N):
                drift=Sigma[j]*np.sum((delta*L[:j+1])/(1+delta*L[:j+1])*rho[j,:j+1]*Sigma[:j+1]*h)
                L[j]= L[j] + drift -0.5*Sigma[j]**2*h + Sigma[j]* np.sqrt(h)*np.sum(U[j,:]*xi[i]) 
    denominator_sum = SwaptionUtilities.Rswap_denominator(delta, np.exp(L))
    rswap = SwaptionUtilities.Rswap(delta, np.exp(L), denominator_sum)
    return delta*max(rswap-K, 0)*denominator_sum*zc

<div style="text-align: center; font-weight: bold; font-size: 100%">"Knock-out swaption" pricing example<div>

In [ ]:
n=int(np.ceil(T0/h))
M=1000
xi=bernoulli(0.5, (n, swaption.N, M))
xi=xi.numpy()
results=pd.DataFrame(columns=["Mean", "Var", "Low", "Up"])

In [ ]:
#Algorithm 1
sample=swaption.swaption(xi, h, swaption_algorithm_1)
results.loc[r"Algorithm 1"] =monte_carlo(sample, 0.95)

In [ ]:
results

<div style="text-align: center; font-weight: bold; font-size: 100%">Variance reduction with antithetic method<div>

In [ ]:
anti_results=pd.DataFrame(columns=["Mean", "Var", "Low", "Up"])

In [ ]:
#Algorithm 1
sample=swaption.swaption_antithetic_method(xi, h, swaption_algorithm_1)
anti_results.loc[r"Algorithm 1"] =monte_carlo(sample, 0.95)

In [ ]:
anti_results